**Variables**

In [3]:
# Titulo del juego:

introduccion= "\nBiennvenid@ a Hundir la Flota Team 5"

# Tablero:

letras = ['A','B','D','E','F','G','H','I','J','K','L','M','N','Ñ','O','P','Q','R','S','T','U','V','W','X','Y','Z']
estilo_tablero = {'punto':' · ', 'agua':' = ', 'barco':' O ','tocado':' X '}

# Jugadores:

class Jugador:
    def __init__(self, name, ndim = 10, tipo = 'jugador'):
        self.name = name
        self.tipo = tipo
        self.disparos = []

        tablero = np.full((2,ndim,ndim),estilo_tablero['punto'])
        self.flota, self.radar = tablero[0], tablero[1]
        self.ndim = ndim
        self.letras = letras[:ndim]

    def mostrar_tablero(self):
        ndim = len(self.flota)
        print()
        for fila in range(ndim):
            if fila == 0:
                print(end = '  ')
                for n in range(2):
                    [print(' ', n, end = ' ') for n in range(1,self.ndim + 1)]
                    print('\t', end = '  ')
            print('\n',letras[fila], end = ' ')
            [print(posicion, end = ' ') for posicion in self.flota[fila]]
            print('\t', letras[fila], end = ' ')
            [print(posicion, end = ' ') for posicion in self.radar[fila]]
        print()

    def check_barcos(self):
        if np.any(self.flota == estilo_tablero['barco']):
            return True
        else:
            return False
    
    def activar_radar(self, flota_enemigo):
        self.radar = np.where(flota_enemigo == estilo_tablero['barco'], estilo_tablero['punto'], flota_enemigo)
        return self.radar

**Menú**

In [ ]:
# Menú principal
def menu_principal():
    menu = 'Selecciona algo:\n\t 1. Empezar a jugar\n\t 2. Créditos\n\t 3. Salir'
    eleccion = input(menu)

    while eleccion.isnumeric() != True or int(eleccion) > 4 or int(eleccion) <0:
        eleccion = input(menu)
        print("error")
    print(eleccion)
    return eleccion

# Función para mostrar instrucciones
def mostrar_instrucciones():
    print(""" 
    - Cada jugador tiene un tablero de 10x10.
    - Los barcos se colocan aleatoriamente en el tablero.
    - Los jugadores se turnan para disparar a una coordenada (fila, columna).
    - El impacto se marcará en el tablero:
        Impacto en barco -> X
        Impacto en Agua -> ~
        Barco hundido -> O
    - El objetivo es hundir todos los barcos del rival.
    - En este Caso el rival será CPU
    - ¡Buena suerte!
    """)

# Función para corregir mayúsculas y minúsculas del nombre
def corregir_nombre(nombre):
    return nombre.strip().lower()  # Elimina espacios y convierte a minúsculas

# Presentación juego y pregunta por nombre
def inicio_juego():
    print(introduccion)

    #Coger el nombre del jugador
    nombre_jugador=input("Para empezar a jugar debes decir tu nombre")
    nombre_jugador = corregir_nombre(nombre_jugador) #Corregimos el nombre como hemos indicado arriba
    print("Bienvenido",nombre_jugador.capitalize()) #Imrpimimos con la primera letra en mayuscula

    print(f"{nombre_jugador}, Estas son las reglas del juego:\n")
    mostrar_instrucciones()
    
    return nombre_jugador

**Colocar barcos**

**Turno de batalla**

In [5]:
def clear(): # Limpiar pantalla
    if os.name == "posix":
        os.system ("clear")
    elif os.name == ("ce", "nt", "dos"):
        os.system ("cls")

## FUNCIONES DE DISPARO ##

def recibir_disparo(tablero, coordenada):
    if tablero[coordenada] == estilo_tablero['barco']:
        tablero[coordenada] = estilo_tablero['tocado']
        return True
    elif tablero[coordenada] == estilo_tablero['tocado']:
        print("Ya has disparado aquí")
        return False
    else:
        tablero[coordenada] = estilo_tablero['agua']
        print("Agua")
        return False

def disparo_coordenadas(tablero, coordenada):    
    fila, columna = coordenada[0], coordenada[1:]
    if len(coordenada) > 3:
        return False
    elif columna.isnumeric() != True:
        return False
    elif fila.upper() not in tablero.letras or int(columna) > tablero.ndim:
        return False
    fila = letras.index(fila.upper())
    columna = int(columna)-1
    return (fila,columna)

def disparo_random(tablero):
    coordenada = (np.random.randint(0,len(tablero)-1),np.random.randint(0,len(tablero)-1))
    while tablero[coordenada] == 'X' or tablero[coordenada] == '~':
        coordenada = (np.random.randint(0,len(tablero)-1),np.random.randint(0,len(tablero)-1))
    recibir_disparo(tablero,coordenada)

## FUNCION DE JUEGO ##

def turno_batalla():
    duo = [jugador,cpu]

    while True:
        atacante, enemigo = duo[0], duo[1]

        if atacante.name == 'vision':
            print(f'Tablero {cpu.name}')
            enemigo.activar_radar(jugador.flota)
            enemigo.mostrar_tablero()

        if atacante.tipo == 'jugador':
            print(f'Turno {atacante.name} \t\t CAÑONAZOS: {atacante.disparos.count(True)}/{len(atacante.disparos)}')
            jugador.activar_radar(cpu.flota)
            jugador.mostrar_tablero()
            
            coordenada = input('Introduce coordenada de disparo: ')
            disparo = disparo_coordenadas(enemigo, coordenada)
            if coordenada == 'salir':
                break
            while disparo == False:
                coordenada = input('Coordenadas incorrectas. Vuelve a intentarlo: ')
                disparo = disparo_coordenadas(enemigo, coordenada)
            acierto = recibir_disparo(enemigo.flota, disparo)
            atacante.disparos.append(acierto)
        else:
            print(f'Turno {atacante.name} \t\t CAÑONAZOS: {atacante.disparos.count(True)}/{len(atacante.disparos)}')
            acierto = disparo_random(enemigo.flota) 
            atacante.disparos.append(acierto)
        
        if acierto == True:
            if enemigo.check_barcos() != True:
                print('Ganaste')
                break
        else:
            duo.reverse()
            continue
    
        clear()    # Limpiamos la pantalla (no funciona en notebooks)
    
    return 'salida'

**MAIN**

In [10]:
import numpy as np

while True:
    eleccion = menu_principal()

    if eleccion == 1:
        nombre_jugador = inicio_juego()

        jugador = Jugador(nombre_jugador)
        cpu = Jugador('CPU','cpu')

        #colocar = colocar_barcos()
        batalla = turno_batalla()
        if batalla == 'salida':
            continue
    elif eleccion == 2:
        print('creditos')
    elif eleccion == 3:
        break

print('quit()')

KeyboardInterrupt: Interrupted by user